In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib  # For model persistence
import datetime  # For model versioning
import warnings
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_excel('Telco_customer_churn.xlsx')
data.head()

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,...,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,...,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,...,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,...,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,86,5372,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,...,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,84,5003,Moved
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,...,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Yes,1,89,5340,Competitor had better devices


In [5]:
data

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,...,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,...,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,...,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,...,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,86,5372,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,...,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,84,5003,Moved
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,...,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Yes,1,89,5340,Competitor had better devices
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,2569-WGERO,1,United States,California,Landers,92285,"34.341737, -116.539416",34.341737,-116.539416,Female,...,Two year,Yes,Bank transfer (automatic),21.15,1419.4,No,0,45,5306,NaN
7039,6840-RESVB,1,United States,California,Adelanto,92301,"34.667815, -117.536183",34.667815,-117.536183,Male,...,One year,Yes,Mailed check,84.80,1990.5,No,0,59,2140,NaN
7040,2234-XADUH,1,United States,California,Amboy,92304,"34.559882, -115.637164",34.559882,-115.637164,Female,...,One year,Yes,Credit card (automatic),103.20,7362.9,No,0,71,5560,NaN
7041,4801-JZAZL,1,United States,California,Angelus Oaks,92305,"34.1678, -116.86433",34.167800,-116.864330,Female,...,Month-to-month,Yes,Electronic check,29.60,346.45,No,0,59,2793,NaN


In [7]:
data.shape

(7043, 33)

In [9]:
data['Churn Label'].unique()

array(['Yes', 'No'], dtype=object)

In [11]:
data.dtypes

CustomerID            object
Count                  int64
Country               object
State                 object
City                  object
Zip Code               int64
Lat Long              object
Latitude             float64
Longitude            float64
Gender                object
Senior Citizen        object
Partner               object
Dependents            object
Tenure Months          int64
Phone Service         object
Multiple Lines        object
Internet Service      object
Online Security       object
Online Backup         object
Device Protection     object
Tech Support          object
Streaming TV          object
Streaming Movies      object
Contract              object
Paperless Billing     object
Payment Method        object
Monthly Charges      float64
Total Charges         object
Churn Label           object
Churn Value            int64
Churn Score            int64
CLTV                   int64
Churn Reason          object
dtype: object

In [13]:
data.drop(['CustomerID', 'Churn Reason', 'Country', 'State', 'Zip Code', 'Lat Long', 'Latitude', 'Longitude', 'Paperless Billing','City', 'Gender', 'Senior Citizen', 'Partner', 'Dependents','Multiple Lines','Online Security', 'Online Backup','Tech Support', 'Streaming TV', 'Streaming Movies'], inplace = True, axis = 1)

In [15]:
data.shape

(7043, 13)

In [17]:
data.isnull().sum()

Count                0
Tenure Months        0
Phone Service        0
Internet Service     0
Device Protection    0
Contract             0
Payment Method       0
Monthly Charges      0
Total Charges        0
Churn Label          0
Churn Value          0
Churn Score          0
CLTV                 0
dtype: int64

In [19]:
data['Churn Label'].unique()

array(['Yes', 'No'], dtype=object)

In [21]:
data.columns

Index(['Count', 'Tenure Months', 'Phone Service', 'Internet Service',
       'Device Protection', 'Contract', 'Payment Method', 'Monthly Charges',
       'Total Charges', 'Churn Label', 'Churn Value', 'Churn Score', 'CLTV'],
      dtype='object')

In [23]:
# Step 2: Data Preprocessing
data.dropna(inplace=True)

# Identify categorical columns[ 'Phone Service','Internet Service', 'Device Protection', 'Contract','Payment Method']
#['CustomerID', 'Churn Reason', 'Country', 'State', 'Zip Code', 'Lat Long', 'Latitude', 'Longitude', 'Paperless Billing']
categorical_cols = ['Phone Service', 'Internet Service','Device Protection', 'Contract','Payment Method']

# Apply one-hot encoding
encoder = OneHotEncoder(drop='first', sparse_output=False)
encoded_cols = pd.DataFrame(encoder.fit_transform(data[categorical_cols]), columns=encoder.get_feature_names_out(categorical_cols))

# Initialize StandardScaler
scaler = StandardScaler()
# Scale the relevant columns
numerical_cols = ['Tenure Months','Monthly Charges','Churn Score','CLTV']
data[numerical_cols] = scaler.fit_transform(data[numerical_cols])


# Drop original categorical columns and add encoded columns
data = data.drop(columns=categorical_cols)
data = pd.concat([data, encoded_cols], axis=1)

In [25]:
data['Total Charges'] = pd.to_numeric(data['Total Charges'], errors='coerce')

In [27]:
X = data.drop('Churn Label', axis=1)  # Your features
y = data['Churn Label']  # Your target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check class distribution in the training and testing sets
print("Training set class distribution:")
print(y_train.value_counts())

print("Test set class distribution:")
print(y_test.value_counts())


Training set class distribution:
Churn Label
No     4165
Yes    1469
Name: count, dtype: int64
Test set class distribution:
Churn Label
No     1009
Yes     400
Name: count, dtype: int64


In [29]:
X.dtypes

Count                                       int64
Tenure Months                             float64
Monthly Charges                           float64
Total Charges                             float64
Churn Value                                 int64
Churn Score                               float64
CLTV                                      float64
Phone Service_Yes                         float64
Internet Service_Fiber optic              float64
Internet Service_No                       float64
Device Protection_No internet service     float64
Device Protection_Yes                     float64
Contract_One year                         float64
Contract_Two year                         float64
Payment Method_Credit card (automatic)    float64
Payment Method_Electronic check           float64
Payment Method_Mailed check               float64
dtype: object

In [31]:
y

0       Yes
1       Yes
2       Yes
3       Yes
4       Yes
       ... 
7038     No
7039     No
7040     No
7041     No
7042     No
Name: Churn Label, Length: 7043, dtype: object

In [33]:
y.nunique()

2

In [35]:
X = X.fillna(method='ffill')  # Forward fill missing values
y = y.fillna(method='ffill')

In [37]:
X.isnull().any()

Count                                     False
Tenure Months                             False
Monthly Charges                           False
Total Charges                             False
Churn Value                               False
Churn Score                               False
CLTV                                      False
Phone Service_Yes                         False
Internet Service_Fiber optic              False
Internet Service_No                       False
Device Protection_No internet service     False
Device Protection_Yes                     False
Contract_One year                         False
Contract_Two year                         False
Payment Method_Credit card (automatic)    False
Payment Method_Electronic check           False
Payment Method_Mailed check               False
dtype: bool

In [39]:
X.dropna()

,Count,Tenure Months,Monthly Charges,Total Charges,Churn Value,Churn Score,CLTV,Phone Service_Yes,Internet Service_Fiber optic,Internet Service_No,Device Protection_No internet service,Device Protection_Yes,Contract_One year,Contract_Two year,Payment Method_Credit card (automatic),Payment Method_Electronic check,Payment Method_Mailed check
0,1,-1.236724,-0.362660,108.15,1,1.268402,-0.981675,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,-1.236724,0.197365,151.65,1,0.385650,-1.436462,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,-0.992402,1.159546,820.50,1,1.268402,0.821409,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,1,-0.177995,1.330711,3046.05,1,1.175481,0.509483,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,1,0.677133,1.294151,5036.30,1,1.407784,0.794358,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,1,1.613701,-1.449476,1419.40,0,-0.636484,0.765617,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
7039,1,-0.340876,0.665992,1990.50,0,0.013965,-1.910691,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
7040,1,1.613701,1.277533,7362.90,0,0.571493,0.980330,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
7041,1,-0.870241,-1.168632,346.45,0,0.013965,-1.358692,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [41]:
X = X.fillna(X.mean())

In [43]:
log = RandomForestClassifier()

log.fit(X_train,y_train)

print(f"Training Score : {log.score(X_train, y_train)}")
print(f"Testing Score : {log.score(X_test, y_test)}")

Training Score : 1.0
Testing Score : 1.0


In [45]:
pred_train=log.predict(X_train)
pred_test=log.predict(X_test)

In [47]:
print(metrics.classification_report(y_train, pred_train))

              precision    recall  f1-score   support

          No       1.00      1.00      1.00      4165
         Yes       1.00      1.00      1.00      1469

    accuracy                           1.00      5634
   macro avg       1.00      1.00      1.00      5634
weighted avg       1.00      1.00      1.00      5634



In [49]:
from sklearn.model_selection import cross_val_score
model = LogisticRegression()
scores = cross_val_score(model, X, y, cv=5)  # 5-fold cross-validation
print(f'Cross-validation scores: {scores}')
print(f'Mean cross-validation score: {scores.mean()}')

Cross-validation scores: [1.         1.         1.         1.         0.99715909]
Mean cross-validation score: 0.9994318181818181
